### Setup and teardown scripts to setup a test area

In [ ]:
import os

check_dir = os.path.basename(os.path.abspath(os.curdir))
if check_dir == 'examples':
    os.chdir('..')

def setup_project_area() -> None:
    if not os.path.exists("tests/temp_data"):
        os.system("wget http://s3df.slac.stanford.edu/people/echarles/xfer/ci_test.tgz -o /dev/null")
        os.system(
            "tar zxvf ci_test.tgz -C tests"
        )

    if not os.path.exists("tests/temp_data/data/test"):
        os.makedirs("tests/temp_data/data/test")
        os.system(
            "wget http://s3df.slac.stanford.edu/people/echarles/xfer/roman_rubin_2023_maglim_25.5_baseline_100k.hdf5 -o /dev/null"
        )
        os.system(
            "mv roman_rubin_2023_maglim_25.5_baseline_100k.hdf5 tests/temp_data/data/test/ci_test_blend_baseline_100k.hdf5"
        )

def teardown_project_area() -> None:
    if not os.environ.get("NO_TEARDOWN"):
        os.system("\\rm -rf tests/temp_data")
        os.system("\\rm -f ci_test.tgz")

setup_project_area()

In [ ]:
from rail.projects import RailProject

project = RailProject.load_config("tests/ci_project.yaml")

In [ ]:
catalog_files_truth = project.get_catalog_files("truth")
print(catalog_files_truth)

In [ ]:
project.reduce_data(
    catalog_template="truth",
    output_catalog_template="reduced",
    reducer_class_name="roman_rubin",
    input_selection="",
    selection="gold",
)


In [ ]:
project.subsample_data(
    catalog_template="reduced",
    file_template="train_file_10",
    subsampler_class_name="random_subsampler",
    subsample_name="train_10",
    flavor="baseline",
    selection="gold",
)

In [ ]:
project.build_pipelines(flavor="baseline")

In [ ]:
single_ceci_command = project.make_pipeline_single_input_command(
    pipeline_name="pz",
    flavor="basline",
    selection="gold",
)
print(single_ceci_command)

In [ ]:
ceci_catalog_commands = project.make_pipeline_catalog_commands(
    pipeline_name="spec_selection",
    flavor="basline",
    selection="gold",
    spec_selections = list(project.get_spec_selections().keys()),
)
print(ceci_catalog_commands)
